In [1]:
# 네이버 해외축구를 통해 자동으로 순위 뽑아내기
from selenium import webdriver
from bs4 import BeautifulSoup
import time

options = webdriver.ChromeOptions()
#options.add_argument('headless') #화면 표시 X
options.add_argument("disable-gpu")   
options.add_argument("lang=ko_KR")    
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')  # user-agent 

driver = webdriver.Chrome('./chromedriver', options=options) # 맥os 일때 경로 지정 중요
driver.get('https://sports.news.naver.com/wfootball/record/index.nhn')

time.sleep(1) # 1초 딜레이

#컨트롤하는 부분 구현
html1=driver.page_source #html1으로 가져오기
soup=BeautifulSoup(html1,'html.parser')

a=soup.find("tbody")
b=a.select("tr")

#print(b)
for i in b:
    c=i.find("span",{"class":"name"}).text # 클래스에 속한 데이터 찾아낼때 사용
    print(c)
driver.close()

리버풀 FC
맨체스터 시티 FC
첼시 FC
레스터 시티 FC
맨체스터 유나이티드 FC
울버햄튼 원더러스 FC
셰필드 유나이티드 FC
아스널 FC
토트넘 홋스퍼 FC
번리 FC
에버턴 FC
사우샘프턴 FC
뉴캐슬 유나이티드 FC
크리스탈 팰리스 FC
브라이튼 앤 호브 알비온 FC
웨스트햄 유나이티드 FC
왓포드 FC
AFC 본머스
아스톤 빌라 FC
노리치 시티 FC


In [76]:
# 지역 , 날짜, 날씨, 최저기온, 최고기온으로 533개를 출력하시오

import json
import pymongo
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request

conn = pymongo.MongoClient('192.168.99.100',32767) #docker 서버 접속
#conn = pymongo.MongoClient('localhost',27017)      #윈도우
db = conn.get_database("db1") # db1생성
table = db.get_collection("weather3") #collection 생성

url = "http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108" # html 주소 가져오기
str = request.urlopen(url) 
soup = BeautifulSoup(str, 'html.parser')
cnt=0;
tmp=soup.select("location")
a1=[]
b1=[]
c1=[]
d1=[]
e1=[]
for i in tmp:
    #print(i)
    #print(i.select_one('city').text)
    tmp1=i.select("data")   
    for j in tmp1:
        a=i.select_one('city').text
        b=j.select_one("tmEf").text
        c=j.select_one("wf").text
        d=j.select_one("tmn").text
        e=j.select_one("tmx").text
        a1.append(a) # 리스트에 저장
        b1.append(b)
        c1.append(c)
        d1.append(d)
        e1.append(e)
#print(a1)
df=pd.DataFrame({"city":a1, "tmEf":b1, "wf":c1, "tmn":d1, "tmx":e1})





#         print("{}, {}, {}, {}, {}".format(i.select_one('city').text,j.select_one("wf").text,j.select_one("tmn").text
#                                  ,j.select_one("tmx").text,j.select_one("tmEf").text))



#타입을 확인한 후  tmef date, tmn int64, tmx int64 로 변환하기
df['city']=df['city'].astype('str') # object= string
df['wf']=df['wf'].astype('str')
df['tmn'] = df['tmn'].astype('int64') # int형 변환하기
df['tmx'] = pd.to_numeric(df['tmx']) # int형 변환하기
df['tmEf'] = pd.to_datetime(df['tmEf'])
df['year'] = df['tmEf'].dt.year
df['month'] = df['tmEf'].dt.month
df['day'] = df['tmEf'].dt.day
df['hour'] = df['tmEf'].dt.hour
df['tmn_tmx'] = (df['tmn']+df['tmx'])/2
#df.info()
x= (df['city'] =='서울') & (df['wf'] == '흐리고 비')
df.loc[x,['city','wf','tmn_tmx']]
# data = df.to_dict(orient='records')  # !!데이터 프레임을 딕셔너리로 변경한 후 추가하기
# table.insert_many(data)

data1 = df.loc[x,['city','wf','tmn_tmx']].to_dict(orient='records')  # !!데이터 프레임을 딕셔너리로 변경한 후 추가하기
table.insert_many(data1)


In [77]:
data1 = table.find({},{'_id':False}) #가져오기

df1 = pd.DataFrame(data1)
df1


,city,wf,tmn_tmx
0,서울,흐리고 비,23.5
1,서울,흐리고 비,23.5
2,서울,흐리고 비,22.5
3,서울,흐리고 비,25.0
4,서울,흐리고 비,25.5


In [5]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request

conn = pymongo.MongoClient('192.168.99.100',32767) #docker 서버 접속
#conn = pymongo.MongoClient('localhost',27017)      #윈도우
db = conn.get_database("db1") # db1생성
table = db.get_collection("weather2") #collection 생성

data = table.find({},{'_id':False}) #가져오기
df2 = pd.DataFrame(data)
df2

# df2.to_csv('weather1.txt', index=False) # csv파일 형태로 저장
# df2.to_csv('test10.csv', index=False) # csv파일 형태로 저장


,city,tmEf,wf,tmn,tmx,year,month,day,hour
0,서울,2020-07-13 00:00:00,흐리고 비,21,26,2020,7,13,0
1,서울,2020-07-13 12:00:00,흐리고 비,21,26,2020,7,13,12
2,서울,2020-07-14 00:00:00,흐리고 비,19,26,2020,7,14,0
3,서울,2020-07-14 12:00:00,흐림,19,26,2020,7,14,12
4,서울,2020-07-15 00:00:00,맑음,19,28,2020,7,15,0
...,...,...,...,...,...,...,...,...,...
528,서귀포,2020-07-17 00:00:00,흐림,22,27,2020,7,17,0
529,서귀포,2020-07-17 12:00:00,흐리고 비,22,27,2020,7,17,12
530,서귀포,2020-07-18 00:00:00,흐리고 비,23,27,2020,7,18,0
531,서귀포,2020-07-19 00:00:00,흐리고 비,24,29,2020,7,19,0


In [6]:
data = pd.read_csv("./test1.csv", encoding='cp949') # csv파일 읽어오기 , 엑셀에서 한글이 꺠질때
data

FileNotFoundError: [Errno 2] No such file or directory: './test1.csv'